In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup 
import time
import pandas as pd
import requests
import sqlite3
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")
driver.implicitly_wait(30)
driver.maximize_window()

def login():
    try:
        driver.get('https://www.finnomena.com/finno-login/')
        time.sleep(1)
        driver.find_element(By.NAME,'email').send_keys('bungaern2545@gmail.com')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
        driver.find_element(By.NAME,'current_password').send_keys('tiwtiwtiw2545')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
    except Exception as e:
        print(e)
login()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9764\1595637895.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")


In [17]:
def getStockDetails(name):
    sections = ['stock-financial-report','stock-financial-ratio']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    quarters = []
    data = []
    for section in sections:
            obj = {}
            keys = []
            headerElements = driver.find_element(By.CSS_SELECTOR,
        f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                keys.append(key)
            contentElements = driver.find_element(By.CSS_SELECTOR, 
                                f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]}"
                obj[key] = values
                data.append(obj)
    return (quarters,data)

def getFinancialDetails(symbol):
    quarterlyOrAnnual,responseData = getStockDetails(name=symbol)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data = {key: data[key] for key in ['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE']}
    data['Period'] = quarterlyOrAnnual
    data['Symbol'] = symbol
    df = pd.DataFrame(data)
    df = df.replace('', '0')
    df = df.iloc[:-1]
    # df = df.drop(df.columns[[3,5,6,8,9,10,11,12,15,16,17,18,19]], axis=1)
    df.iloc[:, :7] = df.iloc[:, :7].applymap(lambda x: x.replace(',', '')).astype(float)
    df = df.replace(0.00, np.nan)
    return df

In [18]:
# detail = getFinancialDetails('WIN')
# detail

In [19]:
# detail.dtypes

In [20]:
def allSymbolset():
    conn = sqlite3.connect('share_V2.sqlite')
    cursor = conn.cursor()
    cursor.execute("""SELECT I.Symbol 
                        FROM Information as I
                        INNER JOIN Market as M ON I.MarketId = M.MarketId
                        WHERE Mname = 'SET'
                        ORDER BY Symbol ASC""")
    result = cursor.fetchall()
    values = [item[0][:-3] for item in result]
    return values

In [21]:
# def FinnomenaDetail():
#     values = allSymbolset()
#     nameNoData = ['AURA', 'COMM', 'TRUE']
#     details_fin = []
#     for name in values:
#         response = requests.get('https://www.finnomena.com/stock/' + name)
#         if (response.status_code == 404) or (name in nameNoData):
#             continue
#         elif len(details_fin) == 0:
#             df1 = getFinancialDetails(symbol=name)
#             details_fin.append(df1)
#         else:
#             df2 = getFinancialDetails(symbol=name)
#             df_old = details_fin[0]
#             df_new = pd.concat([df_old, df2])
#             details_fin[0] = df_new
#     return details_fin[0]

def FinnomenaDetail():
    values = allSymbolset()
    nameNoData = ['AURA', 'COMM', 'TRUE']
    details = []
    for name in values:
        response = requests.get('https://www.finnomena.com/stock/' + name)
        if (name in nameNoData):
            continue
        else:
            df = getFinancialDetails(symbol=name)
            details.append(df)

    df_new = pd.concat(details)
    return df_new
# (response.status_code == 404) or

In [22]:
# allDetail = FinnomenaDetail()
# allDetail

C:\Users\Admin\AppData\Local\Temp\ipykernel_9764\2374899853.py:41: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :7] = df.iloc[:, :7].applymap(lambda x: x.replace(',', '')).astype(float)


KeyError: 'Asset'

In [ ]:
# allDetail.dtypes